### CIF Simulation w/ Peak Indexing

In [ ]:
import montecarlo_peaks as mcpeak
import diffraction_script as diff
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas
import holoviews.plotting.bokeh

from holoviews import opts
from bokeh.models import ColumnDataSource, DataTable, TableColumn, StringFormatter, LabelSet, Div, CustomJS, Slider, Label, Image, ColorBar, LinearColorMapper, Scatter
from bokeh.layouts import row, column
from scipy.interpolate import RegularGridInterpolator
from bokeh.plotting import show, figure
from bokeh.io import output_notebook, push_notebook, show, curdoc, reset_output
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.util.compiler import TypeScript

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

hv.extension('bokeh')

slider_template = """
<div class="bk-slider-parent">
    <div class="bk-slider bk-bs-slider">
        <input type="range" class="bk-slider-input" id="{slider_id}" value="{slider_value}" min="{slider_start}" max="{slider_end}" step="{slider_step}">
        <div class="bk-slider-value">{slider_title}: {slider_value}</div>
    </div>
</div>
"""

def slider_to_html(slider):
    slider_id = slider.id
    slider_value = slider.value
    slider_start = slider.start
    slider_end = slider.end
    slider_step = slider.step
    slider_title = slider.title

    slider_html = slider_template.format(
        slider_id=slider_id,
        slider_value=slider_value,
        slider_start=slider_start,
        slider_end=slider_end,
        slider_step=slider_step,
        slider_title=slider_title,
    )
    return slider_html

def plot_heatmap(data, x_label='X-axis', y_label='Y-axis', z_label='Value', title='Heatmap',
                 font_size=12, tick_size=10, cmap='turbo', width=400, height=400,
                 x_extent=None, y_extent=None, plot_params=None):

    if isinstance(data, np.ndarray):
        if x_extent is None:
            x_extent = (0, data.shape[1])
        if y_extent is None:
            y_extent = (0, data.shape[0])

        X, Y = np.meshgrid(np.linspace(x_extent[0], x_extent[1], data.shape[1]),
                           np.linspace(y_extent[0], y_extent[1], data.shape[0]))
        df = pd.DataFrame({x_label: X.ravel(), y_label: Y.ravel(), z_label: data.ravel()})
    elif isinstance(data, pd.DataFrame):
        df = data.reset_index().melt(id_vars=[data.columns[0]], var_name=x_label, value_name=z_label)
        df = df.rename(columns={data.columns[0]: y_label})
    else:
        raise ValueError('Input data should be a 2D numpy array or a pandas DataFrame.')

    heatmap = df.hvplot.heatmap(x=x_label, 
                                y=y_label, 
                                C=z_label, 
                                colorbar=True, 
                                cmap=cmap, 
                                width=width, 
                                height=height, 
                                title=title)
                                # color_mapper=color_mapper)

    if plot_params is not None:
        xmin, xmax, ymin, ymax = plot_params['xmin'], plot_params['xmax'], plot_params['ymin'], plot_params['ymax']
        heatmap.opts(xlim=(xmin, xmax), ylim=(ymin, ymax))

    heatmap.opts(
        opts.HeatMap(
            xlabel=x_label, 
            ylabel=y_label, 
            colorbar_opts={'orientation': 'vertical', 'location': 'right'},
            fontsize={'title': font_size, 'labels': font_size, 'xticks': tick_size, 'yticks': tick_size}
        )
    )
    
    return heatmap


def plot_hklindex_bokeh(intensity_map, savepath, Mqxy, Mqz, FMiller, plot_params, img_params, BPeak_params, table_width=400, table_height=400):
    resolutionx, qxymax, qzmax, qzmin = imgParams
    hkl_dimension = BPeakParams[2]

    # ------------------------------------------------------------------------------------------------------------------------
    # Here is where the heatmap is created from the plot_heatmap() funciton above.
    x_extent = (-qxymax, qxymax)
    y_extent = (0, qzmax)

    scaleLog = plot_params.get("scaleLog")
    if scaleLog == True:
        intensity_map = np.log(intensity_map + 1)
    
    norm = plot_params.get("norm")
    if norm == True:
        gridMax = intensity_map.max()
        intensity_map = intensity_map/gridMax
    # x_label='$\mathregular{q_{xy}}$ ($\AA^{-1}$)', y_label='$\mathregular{q_z}$ ($\AA^{-1}$)'

    width = int(900)
    height = int(width * 0.67)

    heatmap = plot_heatmap(intensity_map, 
                           x_label='qxy', 
                           y_label='qz', 
                           z_label='Intensity',
                           title=plot_params['header'], 
                           font_size=plot_params['headerfontsize'], 
                           tick_size=plot_params['tickfontsize'],
                           cmap=plot_params['cmap'], 
                           width=width, 
                           height=height, 
                           x_extent=x_extent, 
                           y_extent=y_extent, 
                           plot_params=plot_params)

    # ------------------------------------------------------------------------------------------------------------------------
    # Here is where the (h k l) labels are generated, and their corresponding qxy/qz values are stored.
    Mindexrange = np.linspace(0, hkl_dimension, hkl_dimension+1)
    Mindexrange = Mindexrange.astype('int')
    simuposi = np.zeros([100,2])
    isimuposi = 0
    MaxI = 0
    label_data = []
    hkl_list = []
    qxy_list = []
    qz_list = []

    for h in Mindexrange:
        for k in Mindexrange:
            for l in Mindexrange:
                if Mqxy[h,k,l]<qxymax and Mqz[h,k,l]>qzmin and Mqz[h,k,l]<qzmax:
                    MaxI = np.maximum(FMiller[h,k,l], MaxI)

    for h in Mindexrange:
        for k in Mindexrange:
            for l in Mindexrange:
                if Mqxy[h,k,l]<qxymax and Mqz[h,k,l]>qzmin and Mqz[h,k,l]<qzmax:
                    if FMiller[h,k,l] > plot_params['hklcutoff']*MaxI:
                        simuposi[isimuposi,0] = Mqxy[h,k,l]
                        simuposi[isimuposi,1] = Mqz[h,k,l]
                        isimuposi = isimuposi+1
                        
                        textstr = '('+str(h-hkl_dimension)+','+str(l-hkl_dimension)+','+str(-k+hkl_dimension)+')'
                        hkl_list.append(textstr)
                        qxy_list.append(np.round(Mqxy[h,k,l],decimals = 2))
                        qz_list.append(np.round(Mqz[h,k,l], decimals = 2))
                        label_data.append({'x': Mqxy[h,k,l], 'y': Mqz[h,k,l], 'text': textstr})

    markers = hv.Scatter((simuposi[:, 0], simuposi[:, 1])).opts(size=6, color='red', tools=['hover']) # Create a Scatter plot for markers
    heatmap_with_markers = heatmap * markers # Combine the heatmap and markers

    label_df = pd.DataFrame(label_data) # Create a DataFrame for LabelSet

    label_source = ColumnDataSource(label_df) # Create LabelSet for annotations
    labels = LabelSet(x='x', 
                      y='y', 
                      text='text', 
                      source=label_source, 
                      x_offset=5, 
                      y_offset=5, 
                      render_mode='canvas', 
                      text_color = '#F6E9E6',
                      text_font_size = {'value': '12px'})

    # Create the Bokeh plot with the heatmap, markers, and labels - this merges the markers with the heatmap.
    bokeh_heatmap = hv.renderer('bokeh').get_plot(heatmap_with_markers).state
    bokeh_heatmap.add_layout(labels)

    # Remove gridlines from the heatmap
    bokeh_heatmap.xgrid.grid_line_color = None
    bokeh_heatmap.ygrid.grid_line_color = None
    
    # ------------------------------------------------------------------------------------------------------------------------

    # ------------------------------------------------------------------------------------------------------------------------
    # Here is where the data table is created and populated.
    data_dict = {
    '(h k l)': hkl_list,
    'qxy': qxy_list,
    'qz': qz_list,
    'd(Å)': [''] * len(hkl_list)
    }

    table_columns = [
        TableColumn(field='(h k l)', 
                    title='(h k l)', 
                    formatter=StringFormatter(font_style='bold')),
        TableColumn(field='qxy', title='qxy'),
        TableColumn(field='qz', title='qz'),
        TableColumn(field='d(Å)', title='d(Å)'),
    ]
    
    data_table = DataTable(source=ColumnDataSource(data_dict), 
                       columns=table_columns,
                       autosize_mode = 'fit_columns',
                       width=table_width, 
                       height=table_height, 
                       css_classes=["custom_table"])
    # ------------------------------------------------------------------------------------------------------------------------

    # ------------------------------------------------------------------------------------------------------------------------
    # Here is where the contrast slider widget objects are created and formatted.
    # Add the Div widget with a white background
    div_white_bg = Div(width=table_width, height=table_height - 50, style={'background-color': 'white'})  # Reduce the height

    # Create JavaScript callbacks for the contrast sliders
    js_callback_lower = CustomJS(args=dict(image_renderer=bokeh_heatmap.renderers[0], color_bar=bokeh_heatmap.select_one({'type': ColorBar})), code="""
        var new_value = cb_obj.value;
        var img = image_renderer.glyph;
        img.color_mapper.low = new_value;
        image_renderer.glyph.color_mapper = img.color_mapper;
        
        // Update the color bar
        color_bar.color_mapper.low = new_value;
        color_bar.color_mapper.trigger('change');
    """)

    js_callback_upper = CustomJS(args=dict(image_renderer=bokeh_heatmap.renderers[0], color_bar=bokeh_heatmap.select_one({'type': ColorBar})), code="""
        var new_value = cb_obj.value;
        var img = image_renderer.glyph;
        img.color_mapper.high = new_value;
        image_renderer.glyph.color_mapper = img.color_mapper;
        
        // Update the color bar
        color_bar.color_mapper.high = new_value;
        color_bar.color_mapper.trigger('change');
    """)

    contrast_low = Slider(start=0, end=intensity_map.max(), value=0, step=0.1, title="Low")
    contrast_high = Slider(start=0, end=intensity_map.max(), value=intensity_map.max(), step=0.1, title="High")
    color_mapper = LinearColorMapper(palette="Turbo256", low=contrast_low.value, high=contrast_high.value)

    # -- Callback function to update the heatmap based on contrast scaling sliders.
    def update(attr, old, new):
        color_mapper.low = contrast_low.value
        color_mapper.high = contrast_high.value
        push_notebook()

    # -- Connect the sliders to the update function.
    contrast_low.on_change('value', update)
    contrast_high.on_change('value', update)
    # ------------------------------------------------------------------------------------------------------------------------

    # ------------------------------------------------------------------------------------------------------------------------
    # Interpolate intensity_map to create a smoother heatmap
    x = np.arange(intensity_map.shape[1])
    y = np.arange(intensity_map.shape[0])
    f = RegularGridInterpolator((y, x), intensity_map, method='linear', bounds_error=False, fill_value=None)

    x_new = np.linspace(0, len(x) - 1, len(x) * 10)  # Increase the factor from 5 to 10
    y_new = np.linspace(0, len(y) - 1, len(y) * 10)  # Increase the factor from 5 to 10

    yv, xv = np.meshgrid(y_new, x_new, indexing='ij')
    intensity_map_smooth = f((yv, xv))

    # Get the image glyph from the heatmap
    image_glyph = None
    for r in bokeh_heatmap.renderers:
        if isinstance(r.glyph, Image):
            image_glyph = r.glyph
            break

    # Update the image glyph data source with the intensity_map_smooth
    if image_glyph:
        image_glyph.data_source.data['image'] = [intensity_map_smooth]
    # ------------------------------------------------------------------------------------------------------------------------
    
    # ------------------------------------------------------------------------------------------------------------------------
    # Add custom CSS to set the header background color to white
    css = Div(text="""
    <style>
        .bk-root .slick-header-column {
            background-color: white !important;
        }
    </style>
    """)
    # ------------------------------------------------------------------------------------------------------------------------
    # Here is where the layout generation and formatting occurs.
    # layout = column(row(bokeh_heatmap, contrast_low, contrast_high, div_white_bg), data_table)
    layout = row(column(bokeh_heatmap, contrast_low, contrast_high, div_white_bg), data_table)
    layout.children.insert(0, css) # implement the css white header background 

    # -- Set up the custom CSS.
    custom_css = """
    <style>
        table.index_table {
            border-collapse: collapse;
            font-family: "Lucida Console", Monaco, monospace;
            font-size: 12px;
        }
        table.index_table td {
            border: 1px solid black;
            padding: 2px 4px;
            text-align: center;
        }
    </style>
    """

    # Add the custom CSS style to your HTML head
    bokeh_plot_html = file_html(layout, CDN, "Bokeh Plot")
    bokeh_plot_html = bokeh_plot_html.replace("</head>", f"{custom_css}</head>")

    reset_output()

    show(layout) # display the output, ported out to HTML in your browser with the prescribed formatting conditions.

    return layout

# ---------------------- SETUP PARAMETERS --------------------------- #
# -- Params for Calculating Expected Bragg Reflections -- #
theta_x = (np.pi/4)*2 # use theta_x and theta_y to adjust initial orientation of the sample
theta_y = (np.pi/4)*0 # use theta_x and theta_y to adjust initial orientation of the sample
hkl_dimension = 7 # number of hkl dimensions to calculate (increases comp time significantly)
BPeakParams = [theta_x, theta_y, hkl_dimension]

# -- Crystallite Orientation/Dispersity Params -- #
sigma_theta = 0.02 # sigma_theta, if you need a small number for single crystal, input~0.01, if you need infinity, input~1000
sigma_phi = 100 # sigma_phi, tunes crystallite variance in phi
sigma_r = 0.01 # sigma_r, use this to tune the peak linewidth
crystParams = [sigma_theta, sigma_phi, sigma_r]

# -- Image Params -- #
resolutionx = 300
qxymax = 2
qzmax = 2
qzmin = 0
imgParams = [resolutionx, qxymax, qzmax, qzmin]

print ("Please ensure that POSCAR (.vasp) is exported from CIF in 'Fractional Coordinates'.")

plot_params = {
    "samplename": "MA2DMF2Pb2I6",
    "qxymax": 2, "qzmax": 2,
    "xmin": 0,
    "xmax": 2,
    "ymin": 0,
    "ymax": 2,
    "cmin": 0,
    "cmax": 100,
    "cmap": 'turbo',
    "cblabel": "Intensity",
    "cbsize": 26,
    "scaleLog": False,
    "header": "MA2DMF2Pb2I6 Simulated",
    "headerfontsize": 18,
    "xfontsize": 26,
    "yfontsize": 26,
    "tickfontsize": 26,
    "autosave": True,
    "imgdpi": 500,
    "ext": '.png',
    "colorscale": 0.1,#"linear",
    "fsize": (10,8),#12,
    "hklcutoff": 0.01,
    "norm": True
}

# ---------------------- LOAD DATA --------------------------- #
# -- Load CIF for Simulation -- #
poscar_folder = '/Users/keithwhite/Jupyter Notebooks/giwaxsreduction_main/vasp_files/'
# fileStr = '*MA2DMF2Pb2I6_Petrov2017.*'
fileStr = '*MA2DMF2Pb3I8_Petrov2017.*'
savepath = '/Users/keithwhite/Jupyter Notebooks/giwaxsreduction_main/waxs_sims/'

# ---------------------- RUN SIMULATION --------------------------- #
intensity_map, BPeaks, Mqxy, Mqz, FMiller = diff.intensitymap (poscar_folder, fileStr, BPeakParams, crystParams, imgParams)
plot_hklindex_bokeh(intensity_map, savepath, Mqxy, Mqz, FMiller, plot_params, imgParams, BPeakParams, table_width=500, table_height=400)

### Least Squares Minimization Routine

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
from scipy.signal import fftconvolve
from lmfit import Minimizer, Parameters

# - create three gaussians and one lorentzian to convolute into a single 'convolved' function
# - create each function in a refresh method to reset the function class attribute variables
# for each of these functions
# - loop over the model parameter spaces, storing the iterator variables in the main class
# - each loop iteration should create a set of chi-values that correspond to each set of variables.
# - at the end of each set of iterations, check the value to find the mininum chi from the 
# iterator


def gaussian(x, y, fwhm, center_x, center_y):
    sigma = fwhm / (2 * np.sqrt(2 * np.log(2)))
    return np.exp(-((x - center_x) ** 2 + (y - center_y) ** 2) / (2 * sigma ** 2))

def convoluted_gaussians(params, x, y):
    r0, theta0, phi0 = params['r0'].value, params['theta0'].value, params['phi0'].value
    sigma_r, sigma_theta, sigma_phi = params['sigma_r'].value, params['sigma_theta'].value, params['sigma_phi'].value

    gauss_r = gaussian(x, y, sigma_r, r0, theta0)
    gauss_theta = gaussian(x, y, sigma_theta, theta0, phi0)
    gauss_phi = gaussian(x, y, sigma_phi, r0, phi0)

    return fftconvolve(gauss_r, fftconvolve(gauss_theta, gauss_phi, mode='same'), mode='same')

def residual(params, x, y, data):
    model = convoluted_gaussians(params, x, y)
    return (model - data).ravel()

def fit_heatmap(data, initial_params, return_covar=False):
    if isinstance(data, pd.DataFrame):
        data = data.to_numpy()

    x, y = np.meshgrid(np.arange(data.shape[1]), np.arange(data.shape[0]))

    params = Parameters()
    for key, value in initial_params.items():
        params.add(key, value)

    minimizer = Minimizer(residual, params, fcn_args=(x, y, data))
    result = minimizer.minimize(method='leastsq')

    if return_covar:
        return result.params, result.covar
    else:
        return result.params

In [ ]:
import numpy as np
import scipy.fftpack as fftpack
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
import gc
import time
from collections import defaultdict
from scipy.optimize import minimize

%matplotlib widget

def generate_gaussian_heatmap(height, width, num_gaussians, gaussian_std, amplitude_range):
    heatmap = np.zeros((height, width))
    true_peak_centers = []

    for _ in range(num_gaussians):
        amplitude = np.random.uniform(amplitude_range[0], amplitude_range[1])
        center_y, center_x = np.random.randint(0, height), np.random.randint(0, width)
        true_peak_centers.append((center_y, center_x))
        y, x = np.mgrid[0:height, 0:width]
        heatmap += amplitude * np.exp(-((x - center_x) ** 2 + (y - center_y) ** 2) / (2 * gaussian_std ** 2))

    return heatmap, np.array(true_peak_centers)


def monte_carlo_peak_finding(heatmap, m, n, gradient_threshold):
    height, width = heatmap.shape
    visited = np.zeros_like(heatmap, dtype=bool)

    peak_centers = []

    for _ in range(m):
        start_point = np.random.randint(0, height), np.random.randint(0, width)

        for _ in range(n):
            y, x = start_point
            neighborhood = heatmap[max(y - 1, 0):min(y + 2, height), max(x - 1, 0):min(x + 2, width)]
            next_point = np.unravel_index(np.argmax(neighborhood), neighborhood.shape)
            next_point = next_point[0] + max(y - 1, 0), next_point[1] + max(x - 1, 0)

            if next_point == start_point:
                break
            else:
                start_point = next_point

        y, x = start_point
        if not visited[y, x]:
            peak_centers.append(start_point)

            # Turn off the region around the peak center
            grad_y, grad_x = np.gradient(heatmap)
            gradient_magnitude = np.sqrt(grad_y ** 2 + grad_x ** 2)
            mask = gradient_magnitude < gradient_threshold
            visited[mask] = True

    return np.array(peak_centers)

def fourier_filter_and_find_peaks(heatmap, cutoff_frequency, m, n, gradient_threshold):
    # Apply the Fourier transform to the heatmap
    fft_heatmap = fftpack.fft2(heatmap)

    # Create a low-pass filter in the frequency domain
    rows, cols = heatmap.shape
    crow, ccol = int(rows / 2), int(cols / 2)
    low_pass_filter = np.zeros((rows, cols))
    low_pass_filter[crow - cutoff_frequency:crow + cutoff_frequency, ccol - cutoff_frequency:ccol + cutoff_frequency] = 1

    # Apply the low-pass filter to the heatmap in the frequency domain
    filtered_fft_heatmap = fft_heatmap * low_pass_filter

    # Transform the filtered heatmap back to the spatial domain
    filtered_heatmap = np.real(fftpack.ifft2(filtered_fft_heatmap))

    # Run the Monte Carlo peak finding algorithm on the filtered heatmap
    peak_centers = monte_carlo_peak_finding(filtered_heatmap, m, n, gradient_threshold)

    return peak_centers

def looped_monte_carlo_peak_finding(heatmap, cutoff_frequency, m, n, gradient_threshold, num_iterations, reproducibility_threshold, edge_removal):
    gc.collect()

    def distance(p1, p2):
        return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

    peaks_count = defaultdict(int)
    all_peaks = []

    for _ in range(num_iterations):
        np.random.seed(int(time.time() * 1e6) % 2**32)  # Set seed based on the current time
        peak_centers = fourier_filter_and_find_peaks(heatmap, cutoff_frequency, m, n, gradient_threshold)

        for peak in peak_centers:
            peak = tuple(peak)  # Convert numpy array to tuple
            all_peaks.append(peak)
            for other_peak in all_peaks:
                if distance(peak, other_peak) <= reproducibility_threshold:
                    peaks_count[other_peak] += 1
                    break

    reproducible_peaks = [peak for peak, count in peaks_count.items() if count >= num_iterations / 2]

    # Remove peaks at image edges
    reproducible_peaks = [(x, y) for x, y in reproducible_peaks if edge_removal <= x < heatmap.shape[0] - edge_removal and edge_removal <= y < heatmap.shape[1] - edge_removal]

    return np.array(reproducible_peaks)

def plot_heatmap_and_peaks(heatmap, peak_centers):
    plt.figure()
    plt.imshow(heatmap, cmap='hot', interpolation='nearest')
    
    if peak_centers.size > 0:
        plt.scatter(peak_centers[:, 1], peak_centers[:, 0], c='blue', marker='x', s=50)
    
    plt.colorbar()
    plt.show()

def peak_finding_score(true_peak_centers, detected_peak_centers, tolerance):
    true_peak_centers = set(tuple(p) for p in true_peak_centers)
    detected_peak_centers = set(tuple(p) for p in detected_peak_centers)
    score = 0

    for true_peak in true_peak_centers:
        if not any(np.linalg.norm(np.array(true_peak) - np.array(detected_peak)) <= tolerance for detected_peak in detected_peak_centers):
            score += 1
    
    for detected_peak in detected_peak_centers:
        if not any(np.linalg.norm(np.array(true_peak) - np.array(detected_peak)) <= tolerance for true_peak in true_peak_centers):
            score += 1

    return score

def optimize_parameters(heatmap, true_peak_centers, initial_parameters, bounds):
    def objective_function(parameters):
        cutoff_frequency, m, n, gradient_threshold, num_iterations, reproducibility_threshold, edge_removal = parameters
        m, n, num_iterations, edge_removal = int(m), int(n), int(num_iterations), int(edge_removal)
        detected_peak_centers = looped_monte_carlo_peak_finding(heatmap, cutoff_frequency, m, n, gradient_threshold, num_iterations, reproducibility_threshold, edge_removal)
        score = peak_finding_score(true_peak_centers, detected_peak_centers, tolerance=5)
        return score

    result = minimize(objective_function, initial_parameters, bounds=bounds, method='L-BFGS-B')
    return result.x

# # Example usage
# heatmap = generate_gaussian_heatmap(100, 100, 20, 5, (10, 50))
# cutoff_frequency = 5
# m = 50
# n = 10
# gradient_threshold = 0.1
# num_iterations = 5
# reproducibility_threshold = 5
# edge_removal = 1

# reproducible_peak_centers = looped_monte_carlo_peak_finding(heatmap, cutoff_frequency, m, n, gradient_threshold, num_iterations, reproducibility_threshold, edge_removal)
# print(reproducible_peak_centers)

# plot_heatmap_and_peaks(heatmap, reproducible_peak_centers)

# Example usage
height, width = 100, 100
num_gaussians = 20
gaussian_std = 5
amplitude_range = (10, 50)

heatmap, true_peak_centers = generate_gaussian_heatmap(height, width, num_gaussians, gaussian_std, amplitude_range)

initial_parameters = [5, 50, 10, 0.1, 5, 5, 1]  # Initial values for [cutoff_frequency, m, n, gradient_threshold, num_iterations, reproducibility_threshold, edge_removal]
bounds = [(1, 20), (1, 200), (1, 100), (0.001, 1), (1, 10), (1, 20), (0, 10)]  # Bounds for the parameters

optimized_parameters = optimize_parameters(heatmap, true_peak_centers, initial_parameters, bounds)
print("Optimized parameters:", optimized_parameters)

reproducible_peak_centers = looped_monte_carlo_peak_finding(heatmap, *optimized_parameters)
print(reproducible_peak_centers)

plot_heatmap_and_peaks(heatmap, reproducible_peak_centers)

In [ ]:
import pandas as pd
import os
import glob

def load_and_process_txt_files(directory, output_dir):
    txt_files = glob.glob(os.path.join(directory, "*.txt"))

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    dfs = []

    for file in txt_files:
        filename = os.path.splitext(os.path.basename(file))[0]
        df = pd.read_csv(file, sep=' ', comment='#', header=None)
        df.columns = ['q_' + filename, 'i_' + filename, 'err_' + filename]
        dfs.append(df)

    final_df = pd.concat(dfs, axis=1)
    output_file = os.path.join(output_dir, 'output.csv')
    final_df.to_csv(output_file, index=False)

if __name__ == "__main__":
    input_directory = "/Users/keithwhite/Desktop/1 main project/data/data_saxs/SAXS Data/saxsdata/" # Change this to your input .txt files directory
    output_directory = "/Users/keithwhite/Desktop/1 main project/data/data_saxs/SAXS Data/saxsdata/" # Change this to your desired output directory
    load_and_process_txt_files(input_directory, output_directory)


##### (h k l) Indexing Single Atom Basis - No Forbidden Reflections

In [ ]:
import math
import itertools
import numpy as np
import pandas as pd
import hvplot.pandas
import holoviews as hv
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
hv.extension('bokeh')

class MillerIndex:
    def __init__(self, params):
        self.a, self.b, self.c, self.alpha, self.beta, self.gamma, self.Mhkl = params
        self.params = params

    def crystal_system(self):
        if self.a == self.b == self.c and self.alpha == self.beta == self.gamma == 90:
            return self.cubic
        elif self.a == self.b != self.c and self.alpha == self.beta == 90 and self.gamma == 120:
            return self.hexagonal
        elif self.a == self.b != self.c and self.alpha == self.beta == self.gamma == 90:
            return self.tetragonal
        elif self.a != self.b != self.c and self.alpha == self.beta == self.gamma == 90:
            return self.orthorhombic
        elif self.a == self.b != self.c and self.alpha == self.beta != self.gamma and all(x == 90 for x in [self.alpha, self.beta]):
            return self.monoclinic
        elif self.a != self.b != self.c and self.alpha != self.beta != self.gamma and all(x == 90 for x in [self.alpha, self.beta, self.gamma]):
            return self.triclinic
        else:
            return self.rhombohedral

    def determine_crystal_system(self):
        if self.a == self.b == self.c and self.alpha == self.beta == self.gamma != 90:
            return self.rhombohedral
        elif self.a == self.b != self.c and self.alpha == self.beta == 90 and self.gamma == 120:
            return self.hexagonal
        elif self.a == self.b == self.c and self.alpha == self.beta == self.gamma == 90:
            return self.cubic
        elif self.a != self.b != self.c and self.alpha == self.beta == self.gamma == 90:
            return self.orthorhombic
        elif self.a == self.b != self.c and self.alpha == self.beta == self.gamma == 90:
            return self.tetragonal
        elif self.a != self.b != self.c and self.alpha == self.gamma == 90 and self.beta != 90:
            return self.monoclinic
        else:
            return self.triclinic

    def triclinic(self):
        return self.a, self.b, self.c, self.alpha, self.beta, self.gamma

    def monoclinic(self):
        return self.a, self.b, self.c, 90, self.beta, 90

    def orthorhombic(self):
        return self.a, self.b, self.c, 90, 90, 90

    def tetragonal(self):
        return self.a, self.a, self.c, 90, 90, 90

    def rhombohedral(self):
        return self.a, self.a, self.a, self.alpha, self.alpha, self.alpha

    def hexagonal(self):
        return self.a, self.a, self.c, 90, 90, 120

    def cubic(self):
        return self.a, self.a, self.a, 90, 90, 90

    def calculate_reciprocal_lattice_vectors(self, a, b, c, alpha, beta, gamma):
        # Convert angles to radians
        alpha_rad = math.radians(alpha)
        beta_rad = math.radians(beta)
        gamma_rad = math.radians(gamma)

        # Calculate unit cell volume V
        V = a * b * c * math.sqrt(1 - math.cos(alpha_rad)**2 - math.cos(beta_rad)**2 - math.cos(gamma_rad)**2 + 2 * math.cos(alpha_rad) * math.cos(beta_rad) * math.cos(gamma_rad))

        # Calculate reciprocal lattice vectors
        a_star = [(2 * math.pi / V) * (b * c * math.sin(alpha_rad)), 0, 0]
        b_star = [0, (2 * math.pi / V) * (a * c * math.sin(beta_rad)), 0]
        c_star = [0, 0, (2 * math.pi / V) * (a * b * math.sin(gamma_rad))]

        return a_star, b_star, c_star
    
    def calculate_q_xyz(self, h, k, l, a_star, b_star, c_star):
        q_x = h * a_star[0] + k * b_star[0] + l * c_star[0]
        q_y = h * a_star[1] + k * b_star[1] + l * c_star[1]
        q_z = h * a_star[2] + k * b_star[2] + l * c_star[2]
        return q_x, q_y, q_z

    def reciprocal_coordinates(self):
        crystal_system_func = self.crystal_system()
        a, b, c, alpha, beta, gamma = crystal_system_func()
        a_star, b_star, c_star = self.calculate_reciprocal_lattice_vectors(a, b, c, alpha, beta, gamma)
        
        hkl_combinations = list(itertools.product(range(-self.Mhkl, self.Mhkl + 1), repeat=3))
        reciprocal_coords = []

        for h, k, l in hkl_combinations:
            if (h, k, l) != (0, 0, 0):
                d_hkl = self.calculate_d_hkl(h, k, l, a, b, c, alpha, beta, gamma)
                q_x, q_y, q_z = self.calculate_q_xyz(h, k, l, a_star, b_star, c_star)
                reciprocal_coords.append({'h': h, 'k': k, 'l': l, 'd_hkl': d_hkl, 'q_x': q_x, 'q_y': q_y, 'q_z': q_z})

        return reciprocal_coords

    '''
    # def reciprocal_coordinates(self):
    #     crystal_system_func = self.crystal_system()
    #     a, b, c, alpha, beta, gamma = crystal_system_func()
    #     a_star, b_star, c_star = self.calculate_reciprocal_lattice_vectors(a, b, c, alpha, beta, gamma)

    #     hkl_combinations = list(itertools.product(range(-self.Mhkl, self.Mhkl + 1), repeat=3))
    #     hkl_combinations.remove((0, 0, 0))  # Exclude (0, 0, 0) as it does not give a valid d-spacing

    #     results = []
    #     for h, k, l in hkl_combinations:
    #         qx = h * a_star + k * b_star + l * c_star
    #         qy = h * a_star + k * b_star + l * c_star
    #         qz = h * a_star + k * b_star + l * c_star

    #         results.append({
    #             'h': h,
    #             'k': k,
    #             'l': l,
    #             'q_x': qx,
    #             'q_y': qy,
    #             'q_z': qz,
    #             })

    #     return results
    '''

    def calculate_d_hkl(self, h, k, l, a, b, c, alpha, beta, gamma):
        sin_alpha = np.sin(np.radians(alpha))
        sin_beta = np.sin(np.radians(beta))
        sin_gamma = np.sin(np.radians(gamma))

        if self.crystal_system() == self.cubic:
            return 1 / ((h**2 + k**2 + l**2) / a**2)**0.5
        elif self.crystal_system() == self.hexagonal:
            return 1 / ((4/3 * (h**2 + h*k + k**2) / a**2 + l**2 / c**2)**0.5)
        elif self.crystal_system() == self.tetragonal:
            return 1 / ((h**2 + k**2) / a**2 + l**2 / c**2)**0.5
        elif self.crystal_system() == self.orthorhombic:
            return 1 / ((h**2 / a**2 + k**2 / b**2 + l**2 / c**2)**0.5)
        elif self.crystal_system() == self.monoclinic:
            cos_gamma = np.cos(np.radians(gamma))
            return 1 / ((h**2 / a**2 + k**2 / b**2 + l**2 / c**2 - 2 * h * k * cos_gamma / (a * b))**0.5)
        elif self.crystal_system() == self.triclinic:
            cos_alpha = np.cos(np.radians(alpha))
            cos_beta = np.cos(np.radians(beta))
            cos_gamma = np.cos(np.radians(gamma))
            V = self.calculate_unit_cell_volume(a, b, c, alpha, beta, gamma)
            return 1 / (((h**2 * b**2 * c**2 * sin_alpha**2 + k**2 * a**2 * c**2 * sin_beta**2 + l**2 * a**2 * b**2 * sin_gamma**2 +
                        2 * h * k * a * b * c**2 * cos_alpha * cos_beta + 2 * h * l * a**2 * b * c * cos_alpha * cos_gamma +
                        2 * k * l * a * b**2 * c * cos_beta * cos_gamma) / (a**2 * b**2 * c**2 * (1 - cos_alpha**2 - cos_beta**2 - cos_gamma**2 + 2 * cos_alpha * cos_beta * cos_gamma)))**0.5)
        else:
            cos_alpha = np.cos(np.radians(alpha))
            return 1 / ((h**2 + k**2 + h * k * (1 - cos_alpha) / cos_alpha) / a**2 + l**2 / c**2)**0.5

    def plot_reciprocal_coordinates(self, reciprocal_coords):
        # Create a DataFrame from the reciprocal coordinates
        df = pd.DataFrame(reciprocal_coords)
        df['q_xy'] = np.sqrt(df['q_x'] ** 2 + df['q_y'] ** 2)
        df['hkl'] = df[['h', 'k', 'l']].apply(lambda x: f"({x['h']}{x['k']}{x['l']})", axis=1)

        # Create interactive plots using Hvplot
        plot1 = df.hvplot.scatter(x='q_x', y='q_y', hover_cols=['hkl'], title='q_x vs q_y')
        plot2 = df.hvplot.scatter(x='q_x', y='q_z', hover_cols=['hkl'], title='q_x vs q_z')
        plot3 = df.hvplot.scatter(x='q_xy', y='q_z', hover_cols=['hkl'], title='q_xy vs q_z')

        # Combine the plots and show them in the browser
        combined_plot = (plot1 + plot2 + plot3).cols(3)
        return combined_plot

# # Example usage:
# Example usage:
params = [3, 10, 5, 90, 90, 90, 2]
miller_index = MillerIndex(params)
reciprocal_coords = miller_index.reciprocal_coordinates()
plot = miller_index.plot_reciprocal_coordinates(reciprocal_coords)
hvplot.show(plot)

# for item in reciprocal_coords:
#     print(item)

In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
import panel as pn
import itertools
import math
import hvplot.pandas
import holoviews as hv
import panel as pn
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
hv.extension('bokeh')

# import millerindex as mind

class MillerIndex:
    def __init__(self, params):
        self.a, self.b, self.c, self.alpha, self.beta, self.gamma, self.Mhkl = params
        self.params = params
        # self.Mhkl = self.calculate_miller_limits()


    def crystal_system(self):
        if self.a == self.b == self.c and self.alpha == self.beta == self.gamma == 90:
            return self.cubic
        elif self.a == self.b != self.c and self.alpha == self.beta == 90 and self.gamma == 120:
            return self.hexagonal
        elif self.a == self.b != self.c and self.alpha == self.beta == self.gamma == 90:
            return self.tetragonal
        elif self.a != self.b != self.c and self.alpha == self.beta == self.gamma == 90:
            return self.orthorhombic
        elif self.a == self.b != self.c and self.alpha == self.beta != self.gamma and all(x == 90 for x in [self.alpha, self.beta]):
            return self.monoclinic
        elif self.a != self.b != self.c and self.alpha != self.beta != self.gamma and all(x == 90 for x in [self.alpha, self.beta, self.gamma]):
            return self.triclinic
        else:
            return self.rhombohedral

    def determine_crystal_system(self):
        if self.a == self.b == self.c and self.alpha == self.beta == self.gamma != 90:
            return self.rhombohedral
        elif self.a == self.b != self.c and self.alpha == self.beta == 90 and self.gamma == 120:
            return self.hexagonal
        elif self.a == self.b == self.c and self.alpha == self.beta == self.gamma == 90:
            return self.cubic
        elif self.a != self.b != self.c and self.alpha == self.beta == self.gamma == 90:
            return self.orthorhombic
        elif self.a == self.b != self.c and self.alpha == self.beta == self.gamma == 90:
            return self.tetragonal
        elif self.a != self.b != self.c and self.alpha == self.gamma == 90 and self.beta != 90:
            return self.monoclinic
        else:
            return self.triclinic

    def triclinic(self):
        return self.a, self.b, self.c, self.alpha, self.beta, self.gamma

    def monoclinic(self):
        return self.a, self.b, self.c, 90, self.beta, 90

    def orthorhombic(self):
        return self.a, self.b, self.c, 90, 90, 90

    def tetragonal(self):
        return self.a, self.a, self.c, 90, 90, 90

    def rhombohedral(self):
        return self.a, self.a, self.a, self.alpha, self.alpha, self.alpha

    def hexagonal(self):
        return self.a, self.a, self.c, 90, 90, 120

    def cubic(self):
        return self.a, self.a, self.a, 90, 90, 90

        # Determine the crystal system based on the given parameters
        if len(self.params) == 1:
            return cubic
        elif len(self.params) == 2:
            return hexagonal
        else:
            a, b, c = self.params[:3]
            alpha, beta, gamma = self.params[3:6]
            if alpha == 90 and beta == 90 and gamma == 90:
                if a == b == c:
                    return cubic
                elif a == b != c or a != b == c or a == c != b:
                    return tetragonal
                else:
                    return orthorhombic
            elif alpha == beta == gamma:
                return rhombohedral
            elif alpha == gamma == 90 and beta != 90:
                return monoclinic
            else:
                return triclinic

    def calculate_miller_limits(self):
        # Determine the maximum Miller index for the given parameters
        a, b, c = self.params[:3]
        Mhkl = int(np.sqrt(max(a ** 2 + b ** 2, a ** 2 + c ** 2, b ** 2 + c ** 2)))
        return Mhkl

    def calculate_reciprocal_lattice_vectors(self, a, b, c, alpha, beta, gamma):
        # Convert angles to radians
        alpha_rad = math.radians(alpha)
        beta_rad = math.radians(beta)
        gamma_rad = math.radians(gamma)

        # Calculate unit cell volume V
        V = a * b * c * math.sqrt(1 - math.cos(alpha_rad)**2 - math.cos(beta_rad)**2 - math.cos(gamma_rad)**2 + 2 * math.cos(alpha_rad) * math.cos(beta_rad) * math.cos(gamma_rad))

        # Calculate reciprocal lattice vectors
        a_star = 2 * math.pi / V * np.array([b * c * math.sin(alpha_rad),
                                            a * c * math.sin(beta_rad),
                                            a * b * math.sin(gamma_rad)])
        b_star = 2 * math.pi / V * np.array([c * math.cos(alpha_rad),
                                            -1 * b * math.cos(beta_rad),
                                            a * b * math.cos(gamma_rad) - a * c * math.cos(beta_rad) * math.cos(alpha_rad)])
        c_star = 2 * math.pi / V * np.array([0, 0, 1 / c])

        return a_star, b_star, c_star


    def calculate_q_xyz(self, h, k, l, a_star, b_star, c_star):
        q_x = h * a_star[0] + k * b_star[0] + l * c_star[0]
        q_y = h * a_star[1] + k * b_star[1] + l * c_star[1]
        q_z = h * a_star[2] + k * b_star[2] + l * c_star[2]
        return q_x, q_y, q_z

    def reciprocal_coordinates(self):
        crystal_system_func = self.crystal_system()
        a, b, c, alpha, beta, gamma = crystal_system_func()
        a_star, b_star, c_star = self.calculate_reciprocal_lattice_vectors(a, b, c, alpha, beta, gamma)
        
        hkl_combinations = list(itertools.product(range(-self.Mhkl, self.Mhkl + 1), repeat=3))
        reciprocal_coords = []

        for h, k, l in hkl_combinations:
            if (h, k, l) != (0, 0, 0):
                d_hkl = self.calculate_d_hkl(h, k, l, a, b, c, alpha, beta, gamma)
                q_x, q_y, q_z = self.calculate_q_xyz(h, k, l, a_star, b_star, c_star)
                reciprocal_coords.append({'h': h, 'k': k, 'l': l, 'd_hkl': d_hkl, 'q_x': q_x, 'q_y': q_y, 'q_z': q_z})

        return reciprocal_coords


    def calculate_d_hkl(self, h, k, l, a, b, c, alpha, beta, gamma):
            sin_alpha = np.sin(np.radians(alpha))
            sin_beta = np.sin(np.radians(beta))
            sin_gamma = np.sin(np.radians(gamma))

            if self.crystal_system() == self.cubic:
                return 1 / ((h**2 + k**2 + l**2) / a**2)**0.5
            elif self.crystal_system() == self.hexagonal:
                return 1 / ((4/3 * (h**2 + h*k + k**2) / a**2 + l**2 / c**2)**0.5)
            elif self.crystal_system() == self.tetragonal:
                return 1 / ((h**2 + k**2) / a**2 + l**2 / c**2)**0.5
            elif self.crystal_system() == self.orthorhombic:
                return 1 / ((h**2 / a**2 + k**2 / b**2 + l**2 / c**2)**0.5)
            elif self.crystal_system() == self.monoclinic:
                cos_gamma = np.cos(np.radians(gamma))
                return 1 / ((h**2 / a**2 + k**2 / b**2 + l**2 / c**2 - 2 * h * k * cos_gamma / (a * b))**0.5)
            elif self.crystal_system() == self.triclinic:
                cos_alpha = np.cos(np.radians(alpha))
                cos_beta = np.cos(np.radians(beta))
                cos_gamma = np.cos(np.radians(gamma))
                V = self.calculate_unit_cell_volume(a, b, c, alpha, beta, gamma)
                return 1 / (((h**2 * b**2 * c**2 * sin_alpha**2 + k**2 * a**2 * c**2 * sin_beta**2 + l**2 * a**2 * b**2 * sin_gamma**2 +
                            2 * h * k * a * b * c**2 * cos_alpha * cos_beta + 2 * h * l * a**2 * b * c * cos_alpha * cos_gamma +
                            2 * k * l * a * b**2 * c * cos_beta * cos_gamma) / (a**2 * b**2 * c**2 * (1 - cos_alpha**2 - cos_beta**2 - cos_gamma**2 + 2 * cos_alpha * cos_beta * cos_gamma)))**0.5)
            else:
                cos_alpha = np.cos(np.radians(alpha))
                return 1 / ((h**2 + k**2 + h * k * (1 - cos_alpha) / cos_alpha) / a**2 + l**2 / c**2)**0.5

    def plot_reciprocal_coordinates(self, reciprocal_coords):
        # Create a DataFrame from the reciprocal coordinates
        df = pd.DataFrame(reciprocal_coords)
        df['q_xy'] = np.sqrt(df['q_x'] ** 2 + df['q_y'] ** 2)
        df['hkl'] = df[['h', 'k', 'l']].apply(lambda x: f"({x['h']}{x['k']}{x['l']})", axis=1)

        # Create interactive plots using Hvplot
        plot1 = df.hvplot.scatter(x='q_x', y='q_y', hover_cols=['hkl'], title='q_x vs q_y')
        plot2 = df.hvplot.scatter(x='q_x', y='q_z', hover_cols=['hkl'], title='q_x vs q_z')
        plot3 = df.hvplot.scatter(x='q_xy', y='q_z', hover_cols=['hkl'], title='q_xy vs q_z')

        # Combine the plots and show them in the browser
        combined_plot = pn.Row(plot1 + plot2 + plot3)
        return combined_plot

# Example usage:
params = [3, 10, 5, 90, 90, 90, 2]
miller_index = MillerIndex(params)
reciprocal_coords = miller_index.reciprocal_coordinates()
# plot = miller_index.plot_reciprocal_coordinates(reciprocal_coords)
# hvplot.show(plot)
plot = miller_index.plot_reciprocal_coordinates(reciprocal_coords)
pn.panel(plot)


In [ ]:
import hvplot.pandas
import holoviews as hv
import panel as pn
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
hv.extension('bokeh')

import project_millerindexsim.millerindex as mind

# Example usage:
# params = [3, 10, 5, 90, 90, 90, 2]
params = [1, 1, 12, 90, 90, 60, 2]
miller_index = mind.MillerIndex(params)
reciprocal_coords = miller_index.reciprocal_coordinates()
plot = miller_index.plot_reciprocal_coordinates(reciprocal_coords)
hvplot.show(plot)

In [ ]:
import math
import itertools
import numpy as np
import pandas as pd
import hvplot.pandas
import holoviews as hv
import panel as pn
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
hv.extension('bokeh')

class MillerIndex:
    def __init__(self, params):
        self.a, self.b, self.c, self.alpha, self.beta, self.gamma, self.Mhkl = params
        self.params = params

    def crystal_system(self):
        return self.determine_crystal_system()

    def calculate_unit_cell_volume(self, a, b, c, alpha, beta, gamma):
        alpha_rad = math.radians(alpha)
        beta_rad = math.radians(beta)
        gamma_rad = math.radians(gamma)
        V = a * b * c * math.sqrt(1 - math.cos(alpha_rad)**2 - math.cos(beta_rad)**2 - math.cos(gamma_rad)**2 + 2 * math.cos(alpha_rad) * math.cos(beta_rad) * math.cos(gamma_rad))
        return V

    def calculate_reciprocal_lattice_vectors(self):
        a, b, c, alpha, beta, gamma = self.params

        alpha_rad = math.radians(alpha)
        beta_rad = math.radians(beta)
        gamma_rad = math.radians(gamma)

        V = self.calculate_unit_cell_volume(a, b, c, alpha, beta, gamma)

        a_star = [(2 * math.pi / V) * (b * c * math.sin(alpha_rad)), 0, 0]
        b_star = [0, (2 * math.pi / V) * (a * c * math.sin(beta_rad)), 0]
        c_star = [0, 0, (2 * math.pi / V) * (a * b * math.sin(gamma_rad))]

        return a_star, b_star, c_star

    def generate_reciprocal_coordinates(self):
        a_star, b_star, c_star = self.calculate_reciprocal_lattice_vectors()

        hkl_combinations = list(itertools.product(range(-self.Mhkl, self.Mhkl + 1), repeat=3))
        reciprocal_coords = []

        for h, k, l in hkl_combinations:
            if (h, k, l) != (0, 0, 0):
                q_x = h * a_star[0] + k * b_star[0] + l * c_star[0]
                q_y = h * a_star[1] + k * b_star[1] + l * c_star[1]
                q_z = h * a_star[2] + k * b_star[2] + l * c_star[2]
                reciprocal_coords.append({'h': h, 'k': k, 'l': l, 'q_x': q_x, 'q_y': q_y, 'q_z': q_z})

        return reciprocal_coords

    def plot_reciprocal_coordinates(self, reciprocal_coords):
        df = pd.DataFrame(reciprocal_coords)
        df['q_xy'] = np.sqrt(df['q_x'] ** 2 + df['q_y'] ** 2)
        df['hkl'] = df[['h', 'k', 'l']].apply(lambda x: f"({x['h']}{x['k']}{x['l']})", axis=1)

        plot1 = df

# Main function to execute the script
if __name__ == "__main__":
    # Define the lattice parameters
    lattice_params = (4.0, 4.0, 4.0, 90, 90, 90, 3)  # a, b, c, alpha, beta, gamma, Mhkl

    # Create an instance of the MillerIndex class
    miller_index = MillerIndex(lattice_params)

    # Calculate the reciprocal coordinates
    reciprocal_coords = miller_index.reciprocal_coordinates()

    # Plot the reciprocal coordinates
    plot = miller_index.plot_reciprocal_coordinates(reciprocal_coords)
    hv.save(plot, 'reciprocal_coordinates.html', backend='bokeh')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib widget

class MillerIndex:
    def __init__(self, lattice_params, max_hkl=3):
        self.a, self.b, self.c, self.alpha, self.beta, self.gamma, self.max_hkl = lattice_params
        self.reciprocal_vectors = None

    def reciprocal_coordinates(self):
        # Convert lattice angles to radians
        alpha_rad = np.radians(self.alpha)
        beta_rad = np.radians(self.beta)
        gamma_rad = np.radians(self.gamma)

        # Calculate reciprocal lattice vectors
        V = self.a * self.b * self.c * np.sqrt(1 - np.cos(alpha_rad)**2 - np.cos(beta_rad)**2 - np.cos(gamma_rad)**2 + 2 * np.cos(alpha_rad) * np.cos(beta_rad) * np.cos(gamma_rad))
        a_star = 2 * np.pi * (self.b * self.c * np.sin(alpha_rad)) / V
        b_star = 2 * np.pi * (self.c * self.a * np.sin(beta_rad)) / V
        c_star = 2 * np.pi * (self.a * self.b * np.sin(gamma_rad)) / V

        alpha_star = np.arccos((np.cos(beta_rad) * np.cos(gamma_rad) - np.cos(alpha_rad)) / (np.sin(beta_rad) * np.sin(gamma_rad)))
        beta_star = np.arccos((np.cos(gamma_rad) * np.cos(alpha_rad) - np.cos(beta_rad)) / (np.sin(gamma_rad) * np.sin(alpha_rad)))
        gamma_star = np.arccos((np.cos(alpha_rad) * np.cos(beta_rad) - np.cos(gamma_rad)) / (np.sin(alpha_rad) * np.sin(beta_rad)))

        self.reciprocal_vectors = (a_star, b_star, c_star, alpha_star, beta_star, gamma_star)

        # Generate reciprocal lattice points
        hkl_range = np.arange(-self.max_hkl, self.max_hkl + 1)
        hkl_combinations = np.array(np.meshgrid(hkl_range, hkl_range, hkl_range)).T.reshape(-1, 3)
        return hkl_combinations

    def plot_reciprocal_coordinates(self, reciprocal_coords):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        ax.scatter(reciprocal_coords[:, 0], reciprocal_coords[:, 1], reciprocal_coords[:, 2], marker='o')
        ax.set_xlabel('H')
        ax.set_ylabel('K')
        ax.set_zlabel('L')
        ax.set_title('Reciprocal Lattice Points')
        plt.show()

if __name__ == "__main__":
    lattice_params = (3.0, 10.0, 5.0, 90, 90, 90, 3)  # a, b, c, alpha, beta, gamma, max_hkl
    miller_index = MillerIndex(lattice_params)

    reciprocal_coords = miller_index.reciprocal_coordinates()
    miller_index.plot_reciprocal_coordinates(reciprocal_coords)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objs as go
from plotly.subplots import make_subplots

class MillerIndex:
    def __init__(self, lattice_params):
        self.a, self.b, self.c, self.alpha, self.beta, self.gamma, self.max_hkl = lattice_params
        self.reciprocal_coordinates()

    def reciprocal_coordinates(self):
        # Convert lattice angles to radians
        alpha_rad = np.radians(self.alpha)
        beta_rad = np.radians(self.beta)
        gamma_rad = np.radians(self.gamma)

        # Calculate reciprocal lattice vectors
        V = self.a * self.b * self.c * np.sqrt(1 - np.cos(alpha_rad)**2 - np.cos(beta_rad)**2 - np.cos(gamma_rad)**2 + 2 * np.cos(alpha_rad) * np.cos(beta_rad) * np.cos(gamma_rad))
        a_star = 2 * np.pi * (self.b * self.c * np.sin(alpha_rad)) / V
        b_star = 2 * np.pi * (self.c * self.a * np.sin(beta_rad)) / V
        c_star = 2 * np.pi * (self.a * self.b * np.sin(gamma_rad)) / V

        alpha_star = np.arccos((np.cos(beta_rad) * np.cos(gamma_rad) - np.cos(alpha_rad)) / (np.sin(beta_rad) * np.sin(gamma_rad)))
        beta_star = np.arccos((np.cos(gamma_rad) * np.cos(alpha_rad) - np.cos(beta_rad)) / (np.sin(gamma_rad) * np.sin(alpha_rad)))
        gamma_star = np.arccos((np.cos(alpha_rad) * np.cos(beta_rad) - np.cos(gamma_rad)) / (np.sin(alpha_rad) * np.sin(beta_rad)))

        self.reciprocal_vectors = (a_star, b_star, c_star, alpha_star, beta_star, gamma_star)

        # Generate reciprocal lattice points
        hkl_range = np.arange(-self.max_hkl, self.max_hkl + 1)
        hkl_combinations = np.array(np.meshgrid(hkl_range, hkl_range, hkl_range)).T.reshape(-1, 3)
        return hkl_combinations

    def q_values(self, hkl):
        a_star, b_star, c_star, alpha_star, beta_star, gamma_star = self.reciprocal_vectors

        q_x = hkl[:, 0] * a_star + hkl[:, 1] * b_star * np.cos(gamma_star) + hkl[:, 2] * c_star * np.cos(beta_star)
        q_y = hkl[:, 1] * b_star * np.sin(gamma_star) + hkl[:, 2] * c_star * (np.cos(alpha_star) - np.cos(beta_star) * np.cos(gamma_star)) / np.sin(gamma_star)
        q_z = hkl[:, 2] * c_star * np.sqrt(1 - np.cos(alpha_star)**2 - np.cos(beta_star)**2 - np.cos(gamma_star)**2 + 2 * np.cos(alpha_star) * np.cos(beta_star) * np.cos(gamma_star)) / np.sin(gamma_star)

        return np.column_stack((q_x, q_y, q_z))
    
    def plot_reciprocal_coordinates(self, reciprocal_coords, reciprocal_coords_hkl):
        # Keep only positive q_x, q_y, and q_z values
        positive_mask = np.all(reciprocal_coords >= 0, axis=1)
        reciprocal_coords = reciprocal_coords[positive_mask]
        reciprocal_coords_hkl = reciprocal_coords_hkl[positive_mask]

        hover_texts = [f"({hkl[0]}, {hkl[1]}, {hkl[2]})<br>q_x: {q[0]:.4f}<br>q_y: {q[1]:.4f}<br>q_z: {q[2]:.4f}"
                    for hkl, q in zip(reciprocal_coords_hkl, reciprocal_coords)]

        fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'scatter3d', 'colspan': 2}, None],
                                                [{'type': 'scatter'}, {'type': 'scatter'}]],
                            subplot_titles=("Reciprocal Lattice Points (q_x, q_y, q_z)", "q_xy vs. q_z", "q_x vs. q_y"))

        scatter3d = go.Scatter3d(x=reciprocal_coords[:, 0], y=reciprocal_coords[:, 1], z=reciprocal_coords[:, 2],
                                mode='markers', hovertext=hover_texts, marker=dict(size=4))
        fig.add_trace(scatter3d, row=1, col=1)

        scatter_q_xy_q_z = go.Scatter(x=reciprocal_coords[:, 2], y=np.sqrt(reciprocal_coords[:, 0]**2 + reciprocal_coords[:, 1]**2),
                                    mode='markers', hovertext=hover_texts, marker=dict(size=4))
        fig.add_trace(scatter_q_xy_q_z, row=2, col=1)

        scatter_q_x_q_y = go.Scatter(x=reciprocal_coords[:, 0], y=reciprocal_coords[:, 1],
                                    mode='markers', hovertext=hover_texts, marker=dict(size=4))
        fig.add_trace(scatter_q_x_q_y, row=2, col=2)

        fig.update_layout(scene=dict(xaxis_title='q_x', yaxis_title='q_y', zaxis_title='q_z'),
                 xaxis=dict(title='q_z', domain=[0, 0.45]), xaxis2=dict(title='q_x', domain=[0.55, 1]),
                 yaxis=dict(title='q_xy'), yaxis2=dict(title='q_y'))

        fig.show()

if __name__ == "__main__":
    # lattice_params = (3.0, 10.0, 5.0, 90, 90, 90, 2)  # a, b, c, alpha, beta, gamma, max_hkl
    lattice_params = (1.0, 1.0, 12.0, 90, 90, 60, 2)  # a, b, c, alpha, beta, gamma, max_hkl
    miller_index = MillerIndex(lattice_params)

    reciprocal_coords_hkl = miller_index.reciprocal_coordinates()
    reciprocal_coords_q = miller_index.q_values(reciprocal_coords_hkl)
    miller_index.plot_reciprocal_coordinates(reciprocal_coords_q, reciprocal_coords_hkl)

'''
#     def plot_reciprocal_coordinates(self, reciprocal_coords, reciprocal_coords_hkl):
#             fig = plt.figure()

#             # 3D plot (q_x, q_y, q_z)
#             ax1 = fig.add_subplot(221, projection='3d')
#             ax1.scatter(reciprocal_coords[:, 0], reciprocal_coords[:, 1], reciprocal_coords[:, 2], marker='o')
#             ax1.set_xlabel('q_x')
#             ax1.set_ylabel('q_y')
#             ax1.set_zlabel('q_z')
#             ax1.set_title('Reciprocal Lattice Points (q_x, q_y, q_z)')

#             # 2D plot (q_xy vs. q_z)
#             ax2 = fig.add_subplot(223)
#             ax2.scatter(reciprocal_coords[:, 2], np.sqrt(reciprocal_coords[:, 0]**2 + reciprocal_coords[:, 1]**2), marker='o')
#             ax2.set_xlabel('q_z')
#             ax2.set_ylabel('q_xy')
#             ax2.set_title('q_xy vs. q_z')

#             for i, hkl in enumerate(reciprocal_coords_hkl):
#                 ax2.annotate(f"({hkl[0]}, {hkl[1]}, {hkl[2]})", (reciprocal_coords[i, 2], np.sqrt(reciprocal_coords[i, 0]**2 + reciprocal_coords[i, 1]**2)))

#             # 2D plot (q_x vs. q_y)
#             ax3 = fig.add_subplot(224)
#             ax3.scatter(reciprocal_coords[:, 0], reciprocal_coords[:, 1], marker='o')
#             ax3.set_xlabel('q_x')
#             ax3.set_ylabel('q_y')
#             ax3.set_title('q_x vs. q_y')

#             for i, hkl in enumerate(reciprocal_coords_hkl):
#                 ax3.annotate(f"({hkl[0]}, {hkl[1]}, {hkl[2]})", (reciprocal_coords[i, 0], reciprocal_coords[i, 1]))

#             plt.tight_layout()
#             plt.show()

# if __name__ == "__main__":
#     lattice_params = (4.0, 4.0, 4.0, 90, 90, 90, 3)  # a, b, c, alpha, beta, gamma, max_hkl
#     miller_index = MillerIndex(lattice_params)

#     reciprocal_coords_hkl = miller_index.reciprocal_coordinates()
#     reciprocal_coords_q = miller_index.q_values(reciprocal_coords_hkl)
#     miller_index.plot_reciprocal_coordinates(reciprocal_coords_q, reciprocal_coords_hkl)

# #     def plot_reciprocal_coordinates(self, reciprocal_coords):
# #         fig = plt.figure()
# #         ax = fig.add_subplot(111, projection='3d')

# #         ax.scatter(reciprocal_coords[:, 0], reciprocal_coords[:, 1], reciprocal_coords[:, 2], marker='o')
# #         ax.set_xlabel('q_x')
# #         ax.set_ylabel('q_y')
# #         ax.set_zlabel('q_z')
# #         ax.set_title('Reciprocal Lattice Points (q_x, q_y, q_z)')
# #         plt.show()

# # if __name__ == "__main__":
# #     lattice_params = (3.0, 10.0, 4.0, 90, 90, 90, 2)  # a, b, c, alpha, beta, gamma, max_hkl
# #     miller_index = MillerIndex(lattice_params)

# #     reciprocal_coords_hkl = miller_index.reciprocal_coordinates()
# #     reciprocal_coords_q = miller_index.q_values(reciprocal_coords_hkl)
# #     miller_index.plot_reciprocal_coordinates(reciprocal_coords_q)
'''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_plots(a_star, b_star, c_star, Mhkl):
    Mhkl = int(Mhkl)
    coordinates = []

    for h in range(Mhkl):
        for k in range(Mhkl):
            for l in range(Mhkl):
                qx = h * a_star
                qy = k * b_star
                qz = l * c_star
                coordinates.append((qx, qy, qz))

    coordinates = np.array(coordinates)
    qxy = np.sqrt(coordinates[:, 0] ** 2 + coordinates[:, 1] ** 2)

    # Plot qx vs qy
    plt.figure()
    plt.scatter(coordinates[:, 0], coordinates[:, 1])
    plt.xlabel("qx")
    plt.ylabel("qy")
    plt.title("Plot of qx vs qy")
    plt.grid(True)

    # Plot qx vs qz
    plt.figure()
    plt.scatter(coordinates[:, 0], coordinates[:, 2])
    plt.xlabel("qx")
    plt.ylabel("qz")
    plt.title("Plot of qx vs qz")
    plt.grid(True)

    # Plot qxy vs qz
    plt.figure()
    plt.scatter(qxy, coordinates[:, 2])
    plt.xlabel("qxy")
    plt.ylabel("qz")
    plt.title("Plot of qxy vs qz")
    plt.grid(True)

    plt.show()

a_star = 2 * np.pi / 3
b_star = np.pi / 5
c_star = 2 * np.pi / 5
Mhkl = 3

generate_plots(a_star, b_star, c_star, Mhkl)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def compute_cartesian_vectors(a, b, c, alpha, beta, gamma):
    alpha = np.radians(alpha)
    beta = np.radians(beta)
    gamma = np.radians(gamma)
    
    a1 = np.array([a, 0, 0])
    a2 = np.array([b * np.cos(gamma), b * np.sin(gamma), 0])
    a3 = np.array([c * np.cos(beta),
                   c * (np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma),
                   c * np.sqrt(1 - np.cos(beta)**2 - ((np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma))**2)])
    
    return a1, a2, a3

def compute_volume(a1, a2, a3):
    return np.dot(a1, np.cross(a2, a3))

def compute_reciprocal_vectors(a1, a2, a3, V):
    a_star = 2 * np.pi * np.cross(a2, a3) / V
    b_star = 2 * np.pi * np.cross(a3, a1) / V
    c_star = 2 * np.pi * np.cross(a1, a2) / V
    
    return a_star, b_star, c_star

def generate_plots(a_star, b_star, c_star, Mhkl):
    Mhkl = int(Mhkl)
    coordinates = []

    for h in range(Mhkl):
        for k in range(Mhkl):
            for l in range(Mhkl):
                qx = h * a_star
                qy = k * b_star
                qz = l * c_star
                coordinates.append((qx, qy, qz))

    coordinates = np.array(coordinates)
    qxy = np.sqrt(coordinates[:, 0] ** 2 + coordinates[:, 1] ** 2)

    # Plot qx vs qy
    plt.figure()
    plt.scatter(coordinates[:, 0], coordinates[:, 1])
    plt.xlabel("qx")
    plt.ylabel("qy")
    plt.title("Plot of qx vs qy")
    plt.grid(True)

    # Plot qx vs qz
    plt.figure()
    plt.scatter(coordinates[:, 0], coordinates[:, 2])
    plt.xlabel("qx")
    plt.ylabel("qz")
    plt.title("Plot of qx vs qz")
    plt.grid(True)

    # Plot qxy vs qz
    plt.figure()
    plt.scatter(qxy, coordinates[:, 2])
    plt.xlabel("qxy")
    plt.ylabel("qz")
    plt.title("Plot of qxy vs qz")
    plt.grid(True)

    plt.show()

# Example usage:
a, b, c = 1, 1, 1
alpha, beta, gamma = 90, 90, 90
Mhkl = 1

a1, a2, a3 = compute_cartesian_vectors(a, b, c, alpha, beta, gamma)
V = compute_volume(a1, a2, a3)
a_star, b_star, c_star = compute_reciprocal_vectors(a1, a2, a3, V)
generate_plots(a_star, b_star, c_star, Mhkl)

In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt

class MillerIndices:
    def __init__(self, a, b, c, alpha, beta, gamma, Mhkl):
        self.a = a
        self.b = b
        self.c = c
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.Mhkl = Mhkl

        self.a1, self.a2, self.a3 = self.compute_cartesian_vectors()
        self.V = self.compute_volume()
        self.a_star, self.b_star, self.c_star = self.compute_reciprocal_vectors()

    def compute_cartesian_vectors(self):
        alpha = np.radians(self.alpha)
        beta = np.radians(self.beta)
        gamma = np.radians(self.gamma)

        a1 = np.array([self.a, 0, 0])
        a2 = np.array([self.b * np.cos(gamma), self.b * np.sin(gamma), 0])
        a3 = np.array([self.c * np.cos(beta),
                       self.c * (np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma),
                       self.c * np.sqrt(1 - np.cos(beta)**2 - ((np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma))**2)])

        return a1, a2, a3

    def compute_volume(self):
        return np.dot(self.a1, np.cross(self.a2, self.a3))

    def compute_reciprocal_vectors(self):
        a_star = 2 * np.pi * np.cross(self.a2, self.a3) / self.V
        b_star = 2 * np.pi * np.cross(self.a3, self.a1) / self.V
        c_star = 2 * np.pi * np.cross(self.a1, self.a2) / self.V

        return a_star, b_star, c_star

    def generate_plots(self):
        Mhkl = int(self.Mhkl)
        coordinates = []

        for h in range(Mhkl):
            for k in range(Mhkl):
                for l in range(Mhkl):
                    qx = h * self.a_star
                    qy = k * self.b_star
                    qz = l * self.c_star
                    coordinates.append((qx, qy, qz))

        coordinates = np.array(coordinates)
        qxy = np.sqrt(coordinates[:, 0] ** 2 + coordinates[:, 1] ** 2)

        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        # Plot qx vs qy
        axs[0].scatter(coordinates[:, 0], coordinates[:, 1])
        axs[0].set_xlabel("qx")
        axs[0].set_ylabel("qy")
        axs[0].set_title("Plot of qx vs qy")
        axs[0].grid(True)

        # Plot qx vs qz
        axs[1].scatter(coordinates[:, 0], coordinates[:, 2])
        axs[1].set_xlabel("qx")
        axs[1].set_ylabel("qz")
        axs[1].set_title("Plot of qx vs qz")
        axs[1].grid(True)

        # Plot qxy vs qz
        axs[2].scatter(qxy, coordinates[:, 2])
        axs[2].set_xlabel("qxy")
        axs[2].set_ylabel("qz")
        axs[2].set_title("Plot of qxy vs qz")
        axs[2].grid(True)

        plt.show()

# Example usage:
a, b, c = 3, 10, 5
alpha, beta, gamma = 90, 90, 90
Mhkl = 3

miller_indices = MillerIndices(a, b, c, alpha, beta, gamma, Mhkl)
miller_indices.generate_plots()

    # Commented out example
    # a, b, c = 1, 1, 12
    # alpha, beta, gamma = 90, 90, 120
    # Mhkl = 3

    # miller_indices2 = MillerIndices(a, b, c, alpha, beta, gamma, Mhkl)
    # miller_indices2.generate_plots()


##### Simulate (h k l) Indices

In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

class MillerIndices:
    def __init__(self, a, b, c, alpha, beta, gamma, Mhkl):
        self.a = a
        self.b = b
        self.c = c
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.Mhkl = Mhkl

        self.a1, self.a2, self.a3 = self.compute_cartesian_vectors()
        self.V = self.compute_volume()
        self.a_star, self.b_star, self.c_star = self.compute_reciprocal_vectors()

    def compute_cartesian_vectors(self):
        alpha = np.radians(self.alpha)
        beta = np.radians(self.beta)
        gamma = np.radians(self.gamma)

        a1 = np.array([self.a, 0, 0])
        a2 = np.array([self.b * np.cos(gamma), self.b * np.sin(gamma), 0])
        a3 = np.array([self.c * np.cos(beta),
                       self.c * (np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma),
                       self.c * np.sqrt(1 - np.cos(beta)**2 - ((np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma))**2)])

        return a1, a2, a3

    def compute_volume(self):
        return np.dot(self.a1, np.cross(self.a2, self.a3))

    def compute_reciprocal_vectors(self):
        a_star = 2 * np.pi * np.cross(self.a2, self.a3) / self.V
        b_star = 2 * np.pi * np.cross(self.a3, self.a1) / self.V
        c_star = 2 * np.pi * np.cross(self.a1, self.a2) / self.V

        return a_star, b_star, c_star

    def generate_plots1(self):
        Mhkl = int(self.Mhkl)
        coordinates = []

        for h in range(Mhkl):
            for k in range(Mhkl):
                for l in range(Mhkl):
                    qx = h * self.a_star[0] + k * self.b_star[0] + l * self.c_star[0]
                    qy = h * self.a_star[1] + k * self.b_star[1] + l * self.c_star[1]
                    qz = h * self.a_star[2] + k * self.b_star[2] + l * self.c_star[2]
                    coordinates.append((qx, qy, qz))

        coordinates = np.array(coordinates)
        qxy = np.sqrt(coordinates[:, 0] ** 2 + coordinates[:, 1] ** 2)

        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        # Plot qx vs qy
        axs[0].scatter(coordinates[:, 0], coordinates[:, 1])
        axs[0].set_xlabel("qx")
        axs[0].set_ylabel("qy")
        axs[0].set_title("Plot of qx vs qy")
        axs[0].grid(True)

        # Plot qx vs qz
        axs[1].scatter(coordinates[:, 0], coordinates[:, 2])
        axs[1].set_xlabel("qx")
        axs[1].set_ylabel("qz")
        axs[1].set_title("Plot of qx vs qz")
        axs[1].grid(True)

        # Plot qxy vs qz
        axs[2].scatter(qxy, coordinates[:, 2])
        axs[2].set_xlabel("qxy")
        axs[2].set_ylabel("qz")
        axs[2].set_title("Plot of qxy vs qz")
        axs[2].grid(True)

        plt.show()

    def generate_plots2(self):
        Mhkl = int(self.Mhkl)
        coordinates = []
        hkl_labels = []

        for h in range(Mhkl):
            for k in range(Mhkl):
                for l in range(Mhkl):
                    qx = h * self.a_star[0] + k * self.b_star[0] + l * self.c_star[0]
                    qy = h * self.a_star[1] + k * self.b_star[1] + l * self.c_star[1]
                    qz = h * self.a_star[2] + k * self.b_star[2] + l * self.c_star[2]
                    coordinates.append((qx, qy, qz))
                    hkl_labels.append(f"({h}, {k}, {l})")

        coordinates = np.array(coordinates)
        qxy = np.sqrt(coordinates[:, 0] ** 2 + coordinates[:, 1] ** 2)

        # Create a subplot with 1 row and 3 columns
        fig = make_subplots(rows=1, cols=3, subplot_titles=("Plot of qx vs qy", "Plot of qx vs qz", "Plot of qxy vs qz"))

        # Plot qx vs qy
        fig.add_trace(go.Scatter(x=coordinates[:, 0], y=coordinates[:, 1], text=hkl_labels, mode="markers+text", marker=dict(size=8), textposition="top center", name="qx vs qy"), row=1, col=1)

        # Plot qx vs qz
        fig.add_trace(go.Scatter(x=coordinates[:, 0], y=coordinates[:, 2], text=hkl_labels, mode="markers+text", marker=dict(size=8), textposition="top center", name="qx vs qz"), row=1, col=2)

        # Plot qxy vs qz
        fig.add_trace(go.Scatter(x=qxy, y=coordinates[:, 2], text=hkl_labels, mode="markers+text", marker=dict(size=8), textposition="top center", name="qxy vs qz"), row=1, col=3)

        # Update xaxis and yaxis labels
        fig.update_xaxes(title_text="qx", row=1, col=1)
        fig.update_yaxes(title_text="qy", row=1, col=1)
        fig.update_xaxes(title_text="qx", row=1, col=2)
        fig.update_yaxes(title_text="qz", row=1, col=2)
        fig.update_xaxes(title_text="qxy", row=1, col=3)
        fig.update_yaxes(title_text="qz", row=1, col=3)

        fig.show()

    def generate_plots(self):
        Mhkl = int(self.Mhkl)
        coordinates = []
        hkl_labels = []

        for h in range(Mhkl):
            for k in range(Mhkl):
                for l in range(Mhkl):
                    qx = h * self.a_star[0] + k * self.b_star[0] + l * self.c_star[0]
                    qy = h * self.a_star[1] + k * self.b_star[1] + l * self.c_star[1]
                    qz = h * self.a_star[2] + k * self.b_star[2] + l * self.c_star[2]
                    coordinates.append((qx, qy, qz))
                    hkl_labels.append(f"({h}, {k}, {l})")

        coordinates = np.array(coordinates)
        qxy = np.sqrt(coordinates[:, 0] ** 2 + coordinates[:, 1] ** 2)

        # Create a subplot with 1 row and 3 columns
        fig = make_subplots(rows=1, cols=3, subplot_titles=("Plot of qx vs qy", "Plot of qx vs qz", "Plot of qxy vs qz"))

        # Plot qx vs qy
        fig.add_trace(go.Scatter(x=coordinates[:, 0], y=coordinates[:, 1], text=hkl_labels, mode="markers", marker=dict(size=8), hovertemplate="(%{x:.2f}, %{y:.2f})<br>%{text}"), row=1, col=1)

        # Plot qx vs qz
        fig.add_trace(go.Scatter(x=coordinates[:, 0], y=coordinates[:, 2], text=hkl_labels, mode="markers", marker=dict(size=8), hovertemplate="(%{x:.2f}, %{y:.2f})<br>%{text}"), row=1, col=2)

        # Plot qxy vs qz
        fig.add_trace(go.Scatter(x=qxy, y=coordinates[:, 2], text=hkl_labels, mode="markers", marker=dict(size=8), hovertemplate="(%{x:.2f}, %{y:.2f})<br>%{text}"), row=1, col=3)

        # Update xaxis and yaxis labels
        fig.update_xaxes(title_text="qx", row=1, col=1)
        fig.update_yaxes(title_text="qy", row=1, col=1)
        fig.update_xaxes(title_text="qx", row=1, col=2)
        fig.update_yaxes(title_text="qz", row=1, col=2)
        fig.update_xaxes(title_text="qxy", row=1, col=3)
        fig.update_yaxes(title_text="qz", row=1, col=3)

        fig.show()
# Example usage:
# a, b, c = 3, 10, 5
# alpha, beta, gamma = 90, 90, 90
# Mhkl = 3

# miller_indices = MillerIndices(a, b, c, alpha, beta, gamma, Mhkl)
# miller_indices.generate_plots()

# Commented out example
a, b, c = 1, 1, 12
alpha, beta, gamma = 90, 90, 120
Mhkl = 3

miller_indices2 = MillerIndices(a, b, c, alpha, beta, gamma, Mhkl)
miller_indices2.generate_plots()


##### Simulate (h k l) Indices w/ Crystallite Rotations about Peak Center

In [69]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import webbrowser
import tempfile
import sys, os, subprocess

class MillerIndices:
    def __init__(self, a, b, c, alpha, beta, gamma, Mhkl):
        self.a = a
        self.b = b
        self.c = c
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.Mhkl = Mhkl

        self.a1, self.a2, self.a3 = self.compute_cartesian_vectors()
        self.V = self.compute_volume()
        self.a_star, self.b_star, self.c_star = self.compute_reciprocal_vectors()
        self.points = self.get_coordinates()
        self.rotated_points = None

    def compute_cartesian_vectors(self):
        alpha = np.radians(self.alpha)
        beta = np.radians(self.beta)
        gamma = np.radians(self.gamma)

        a1 = np.array([self.a, 0, 0])
        a2 = np.array([self.b * np.cos(gamma), self.b * np.sin(gamma), 0])
        a3 = np.array([self.c * np.cos(beta),
                       self.c * (np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma),
                       self.c * np.sqrt(1 - np.cos(beta)**2 - ((np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma))**2)])

        return a1, a2, a3

    def compute_volume(self):
        return np.dot(self.a1, np.cross(self.a2, self.a3))

    def compute_reciprocal_vectors(self):
        a_star = 2 * np.pi * np.cross(self.a2, self.a3) / self.V
        b_star = 2 * np.pi * np.cross(self.a3, self.a1) / self.V
        c_star = 2 * np.pi * np.cross(self.a1, self.a2) / self.V

        return a_star, b_star, c_star

    def get_coordinates(self):
        Mhkl = int(self.Mhkl)
        coordinates = []

        for h in range(Mhkl):
            for k in range(Mhkl):
                for l in range(Mhkl):
                    qx = h * self.a_star[0] + k * self.b_star[0] + l * self.c_star[0]
                    qy = h * self.a_star[1] + k * self.b_star[1] + l * self.c_star[1]
                    qz = h * self.a_star[2] + k * self.b_star[2] + l * self.c_star[2]
                    coordinates.append((qx, qy, qz))

        return np.array(coordinates)
    
    def generate_lattice_points(self):
        points = []
        for h in range(self.Mhkl + 1):
            for k in range(self.Mhkl + 1):
                for l in range(self.Mhkl + 1):
                    points.append([h, k, l])
        return np.array(points)

    @staticmethod
    def cartesian_to_spherical(x, y, z):
        r = np.sqrt(x**2 + y**2 + z**2)
        theta = np.arccos(z / r)
        phi = np.arctan2(y, x)
        return theta, phi, r

    @staticmethod
    def rotate_theta(theta, phi, r, d_theta):
        theta_rotated = theta + d_theta
        x_rotated = r * np.sin(theta_rotated) * np.cos(phi)
        y_rotated = r * np.sin(theta_rotated) * np.sin(phi)
        z_rotated = r * np.cos(theta_rotated)
        return x_rotated, y_rotated, z_rotated

    @staticmethod
    def rotate_phi(theta, phi, r, d_phi):
        phi_rotated = phi + d_phi
        x_rotated = r * np.sin(theta) * np.cos(phi_rotated)
        y_rotated = r * np.sin(theta) * np.sin(phi_rotated)
        z_rotated = r * np.cos(theta)
        return x_rotated, y_rotated, z_rotated

    def rotate_x(self, thetax):
        thetax = np.radians(thetax)
        Rx = np.array([[1, 0, 0],
                       [0, np.cos(thetax), -np.sin(thetax)],
                       [0, np.sin(thetax), np.cos(thetax)]])
        rotated_coordinates = self.get_coordinates().dot(Rx.T)
        q_xy = np.sqrt(rotated_coordinates[:, 0] ** 2 + rotated_coordinates[:, 1] ** 2)
        return rotated_coordinates[:, 0], rotated_coordinates[:, 1], rotated_coordinates[:, 2], q_xy

    def rotate_y(self, thetay):
        thetay = np.radians(thetay)
        Ry = np.array([[np.cos(thetay), 0, np.sin(thetay)],
                       [0, 1, 0],
                       [-np.sin(thetay), 0, np.cos(thetay)]])
        rotated_coordinates = self.get_coordinates().dot(Ry.T)
        q_xy = np.sqrt(rotated_coordinates[:, 0] ** 2 + rotated_coordinates[:, 1] ** 2)
        return rotated_coordinates[:, 0], rotated_coordinates[:, 1], rotated_coordinates[:, 2], q_xy

    def cart_to_spherical(self, points):
        spherical_points = []
        for point in points:
            r = np.linalg.norm(point)
            theta = np.arccos(point[2] / r)
            phi = np.arctan2(point[1], point[0])
            spherical_points.append(np.array([r, theta, phi]))
        return np.array(spherical_points)

    def spherical_to_cart(self, spherical_point):
        r, theta, phi = spherical_point
        x = r * np.sin(theta) * np.cos(phi)
        y = r * np.sin(theta) * np.sin(phi)
        z = r * np.cos(theta)
        return np.array([x, y, z])
    
    def apply_gaussian_rotations(self, n, theta_mean, phi_mean, sigma_theta, sigma_phi):
        self.rotated_points = []

        spherical_points = self.cart_to_spherical(self.points)

        for point in spherical_points:
            if np.all(point == 0):  # Exclude the (0, 0, 0) point
                continue

            for _ in range(n):
                theta_offset = np.random.normal(theta_mean, sigma_theta)
                phi_offset = np.random.normal(phi_mean, sigma_phi)

                rotated_spherical = point.copy()
                rotated_spherical[1] += theta_offset
                rotated_spherical[2] += phi_offset

                rotated_cartesian = self.spherical_to_cart(rotated_spherical)

                if rotated_cartesian[0] >= 0 and rotated_cartesian[1] >= 0 and rotated_cartesian[2] >= 0:  # Positive quadrant only
                    qxy = np.sqrt(rotated_cartesian[0]**2 + rotated_cartesian[1]**2)
                    self.rotated_points.append(np.hstack([rotated_cartesian, qxy]))

        self.rotated_points = np.array(self.rotated_points)

    def generate_plots_mpl(self):
        if self.rotated_points is None:
            raise ValueError("No rotated points found. Please call apply_gaussian_rotations first.")

        fig, axs = plt.subplots(1, 3, figsize=(18, 6))

        axs[0].scatter(self.rotated_points[:, 0], self.rotated_points[:, 1], label="q_x vs q_y")
        axs[1].scatter(self.rotated_points[:, 0], self.rotated_points[:, 2], label="q_x vs q_z")
        axs[2].scatter(self.rotated_points[:, 1], self.rotated_points[:, 2], label="q_y vs q_z")

        axs[0].set_xlabel('q_x')
        axs[0].set_ylabel('q_y')
        axs[0].legend()

        axs[1].set_xlabel('q_x')
        axs[1].set_ylabel('q_z')
        axs[1].legend()

        axs[2].set_xlabel('q_y')
        axs[2].set_ylabel('q_z')
        axs[2].legend()

        plt.show()

    def generate_2d_plot(self, x, y, x_label, y_label, color):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, mode='markers', marker=dict(size=3, color=color), hoverinfo='x+y'))
        fig.update_layout(xaxis_title=x_label, yaxis_title=y_label, xaxis_title_font=dict(size=14), yaxis_title_font=dict(size=14), width=400, height=400)
        return fig

    def generate_3d_plot(self):
        # Calculate radial distances for color scaling
        radial_distances = np.sqrt(self.rotated_points[:, 0]**2 + self.rotated_points[:, 1]**2 + self.rotated_points[:, 2]**2)
        colors = radial_distances / np.max(radial_distances)

        # Create the 3D plot
        fig = go.Figure()

        fig.add_trace(go.Scatter3d(x=self.rotated_points[:, 0],
                                   y=self.rotated_points[:, 1],
                                   z=self.rotated_points[:, 2],
                                   mode='markers',
                                   marker=dict(size=2, color=colors, colorscale='turbo', showscale=False),
                                   hoverinfo='x+y+z'))

        fig.update_layout(scene=dict(xaxis_title='q_x',
                                      yaxis_title='q_y',
                                      zaxis_title='q_z',
                                      xaxis_title_font=dict(size=14),
                                      yaxis_title_font=dict(size=14),
                                      zaxis_title_font=dict(size=14)),
                          width=1200, height=1000)

        return fig
    
    def generate_plots(self):
        fig_2d = []
        fig_2d.append(self.generate_2d_plot(self.rotated_points[:, 0], self.rotated_points[:, 1], 'q_x', 'q_y', 'red'))
        fig_2d.append(self.generate_2d_plot(self.rotated_points[:, 0], self.rotated_points[:, 2], 'q_x', 'q_z', 'blue'))
        fig_2d.append(self.generate_2d_plot(np.sqrt(self.rotated_points[:, 0]**2 + self.rotated_points[:, 1]**2),
                                            self.rotated_points[:, 2], 'q_xy', 'q_z', 'green'))

        # Create 3D plot
        fig_3d = self.generate_3d_plot()

        # Combine 2D and 3D plots in a single layout
        final_fig = make_subplots(rows=2, cols=3, row_heights=[0.2, 0.65], vertical_spacing=0.1,
                                specs=[[{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}],
                                        [{"type": "scatter3d", "colspan": 3}, None, None]],
                                subplot_titles=("q_x vs q_y", "q_x vs q_z", "q_xy vs q_z", "3D Plot"))

        for idx, fig in enumerate(fig_2d):
            final_fig.add_trace(fig.data[0], row=1, col=idx + 1)
            final_fig.update_xaxes(title_text=fig.layout.xaxis.title.text, row=1, col=idx + 1)
            final_fig.update_yaxes(title_text=fig.layout.yaxis.title.text, row=1, col=idx + 1)

        final_fig.add_trace(fig_3d.data[0], row=2, col=1)

        final_fig.update_layout(scene=dict(xaxis=dict(title='q_x', title_font=dict(size=14)),
                                        yaxis=dict(title='q_y', title_font=dict(size=14)),
                                        zaxis=dict(title='q_z', title_font=dict(size=14))))

        final_fig.update_layout(title="Rotated Points Plots", height=900, width=1200)
        # final_fig.show()
    
        # Convert the figure to an HTML string
        html_string = pio.to_html(final_fig, full_html=True)

        # Save the HTML string to a temporary file and open it in a browser
        with tempfile.NamedTemporaryFile('w', delete=False, suffix='.html') as f:
            f.write(html_string)
            
            # Open the file in Chrome
            try:
                chrome_path = None

                # Locate the Chrome browser based on the operating system
                if sys.platform.startswith('win'):
                    chrome_path = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe'
                elif sys.platform.startswith('darwin'):
                    chrome_path = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
                elif sys.platform.startswith('linux'):
                    chrome_path = '/usr/bin/google-chrome'

                if chrome_path:
                    subprocess.Popen([chrome_path, f.name])
                else:
                    subprocess.Popen(["start", f.name], shell=True)

            except Exception as e:
                print(f"Error opening browser: {e}")
                print("Opening in the default browser instead.")
                subprocess.Popen(["start", f.name], shell=True)

# ---- Generate plots example
lattice = MillerIndices(a=3, b=10, c=15, alpha=90, beta=90, gamma=90, Mhkl=6)
lattice.apply_gaussian_rotations(n=100, theta_mean=0, phi_mean=0, sigma_theta=.1, sigma_phi=0.1)
lattice.generate_plots()

# Each random rotation should have all the equilvalent reflections calculated for that rotation. For instance, in a single 
# rotation, we are rotating the crystallite. So there should be some set of (h k l) values that connect to each other.
# We can't rotate all (h k l) values independently. (h k l) values for a single orientation should be calculated together.
# Each of the 'n' rotations should pertain to one complete set of the full extent of (h k l) values.

/var/folders/5h/c94dty5j5lbgtxxpsfrx5k2c0000gn/T/ipykernel_42353/2087835894.py:116: RuntimeWarning:

invalid value encountered in scalar divide



[50369:259:0502/205347.214533:ERROR:chrome_browser_main.cc(1019)] The use of Rosetta to run the x64 version of Chromium on Arm is neither tested nor maintained, and unexpected behavior will likely result. Please check that all tools that spawn Chromium are Arm-native.


Opening in existing browser session.


In [ ]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# Generate random points
np.random.seed(42)
points = np.random.rand(50, 3)

# Create a 3D scatter plot
scatter = go.Scatter3d(x=points[:, 0], y=points[:, 1], z=points[:, 2], mode='markers', marker=dict(size=5))

# Initial plane parameters
a, b, c, d = 1, -1, 1, 0

# Function to update the plane
def update_plane(a, b, c, d):
    x = np.linspace(0, 1, 10)
    y = np.linspace(0, 1, 10)
    X, Y = np.meshgrid(x, y)
    Z = (-a * X - b * Y - d) / c
    
    return go.Surface(x=X, y=Y, z=Z, opacity=0.5, showscale=False)

plane = update_plane(a, b, c, d)

# Create the initial 3D plot
fig_3d = go.FigureWidget(data=[scatter, plane])
fig_3d.layout.title = "3D Plot with Plane"

# Create the 2D scatter plot for intersection points
fig_2d = go.FigureWidget(data=[go.Scatter(x=[], y=[], mode='markers')])
fig_2d.layout.title = "Intersection Points"
fig_2d.layout.xaxis.title = 'X'
fig_2d.layout.yaxis.title = 'Y'

# Function to update the plot with new plane parameters
def update(change):
    a, b, c, d = slider_a.value, slider_b.value, slider_c.value, slider_d.value
    plane_thickness = slider_thickness.value
    new_plane = update_plane(a, b, c, d)
    fig_3d.data[1].x = new_plane.x
    fig_3d.data[1].y = new_plane.y
    fig_3d.data[1].z = new_plane.z

    # Calculate intersection points
    plane_distance = np.abs(a * points[:, 0] + b * points[:, 1] + c * points[:, 2] + d) / np.sqrt(a**2 + b**2 + c**2)
    intersection_points = points[plane_distance <= plane_thickness]

    # Update the scatter plot with intersection points
    fig_2d.data[0].x = intersection_points[:, 0]
    fig_2d.data[0].y = intersection_points[:, 1]

# Create sliders for plane parameters
slider_a = widgets.FloatSlider(min=-1, max=1, step=0.01, value=a, description='a')
slider_b = widgets.FloatSlider(min=-1, max=1, step=0.01, value=b, description='b')
slider_c = widgets.FloatSlider(min=-1, max=1, step=0.01, value=c, description='c')
slider_d = widgets.FloatSlider(min=-1, max=1, step=0.01, value=d, description='d')
slider_thickness = widgets.FloatSlider(min=0.01, max=0.5, step=0.01, value=0.05, description='Thickness')

# Add the observer to the sliders
slider_a.observe(update, names='value')
slider_b.observe(update, names='value')
slider_c.observe(update, names='value')
slider_d.observe(update, names='value')
slider_thickness.observe(update, names='value')

# Display the interactive plot and sliders
display(widgets.HBox([fig_3d, fig_2d]))
display(slider_a, slider_b, slider_c, slider_d, slider_thickness)


In [1]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# Generate random points
np.random.seed(42)
points = np.random.rand(50, 3)

# Create a 3D scatter plot
scatter = go.Scatter3d(x=points[:, 0], y=points[:, 1], z=points[:, 2], mode='markers', marker=dict(size=5))

# Initial plane parameters
a, b, c, d = 1, -1, 1, 0

# Function to update the plane
def update_plane(a, b, c, d):
    x = np.linspace(0, 1, 10)
    y = np.linspace(0, 1, 10)
    X, Y = np.meshgrid(x, y)
    Z = (-a * X - b * Y - d) / c
    
    return go.Surface(x=X, y=Y, z=Z, opacity=0.5, showscale=False)

plane = update_plane(a, b, c, d)

# Create the initial 3D plot
fig_3d = go.FigureWidget(data=[scatter, plane])
fig_3d.layout.title = "3D Plot with Plane"

# Create the 2D scatter plot for intersection points
fig_2d = go.FigureWidget(data=[go.Scatter(x=[], y=[], mode='markers')])
fig_2d.layout.title = "Intersection Points"
fig_2d.layout.xaxis.title = 'X'
fig_2d.layout.yaxis.title = 'Y'

# Function to update the plot with new plane parameters
def update(change):
    a, b, c, d = slider_a.value, slider_b.value, slider_c.value, slider_d.value
    plane_thickness = slider_thickness.value
    new_plane = update_plane(a, b, c, d)
    fig_3d.data[1].x = new_plane.x
    fig_3d.data[1].y = new_plane.y
    fig_3d.data[1].z = new_plane.z

    # Calculate intersection points
    plane_distance = np.abs(a * points[:, 0] + b * points[:, 1] + c * points[:, 2] + d) / np.sqrt(a**2 + b**2 + c**2)
    intersection_points = points[plane_distance <= plane_thickness]

    # Update the scatter plot with intersection points
    fig_2d.data[0].x = intersection_points[:, 0]
    fig_2d.data[0].y = intersection_points[:, 1]

# Create sliders for plane parameters
slider_a = widgets.FloatSlider(min=-1, max=1, step=0.01, value=a, description='a')
slider_b = widgets.FloatSlider(min=-1, max=1, step=0.01, value=b, description='b')
slider_c = widgets.FloatSlider(min=-1, max=1, step=0.01, value=c, description='c')
slider_d = widgets.FloatSlider(min=-1, max=1, step=0.01, value=d, description='d')
slider_thickness = widgets.FloatSlider(min=0.01, max=0.5, step=0.01, value=0.05, description='Thickness')

# Add the observer to the sliders
slider_a.observe(update, names='value')
slider_b.observe(update, names='value')
slider_c.observe(update, names='value')
slider_d.observe(update, names='value')
slider_thickness.observe(update, names='value')

# Display the interactive plot and sliders
display(widgets.HBox([fig_3d, fig_2d]))
display(slider_a, slider_b, slider_c, slider_d, slider_thickness)

    'data': [{'marker': {'size': 5},
              'mode': 'markers',
          …

FloatSlider(value=1.0, description='a', max=1.0, min=-1.0, step=0.01)

FloatSlider(value=-1.0, description='b', max=1.0, min=-1.0, step=0.01)

FloatSlider(value=1.0, description='c', max=1.0, min=-1.0, step=0.01)

FloatSlider(value=0.0, description='d', max=1.0, min=-1.0, step=0.01)

FloatSlider(value=0.05, description='Thickness', max=0.5, min=0.01, step=0.01)